# 5.3 사전 훈련된 컨브넷 사용하기

### 사전 훈련된 네트워크(pretrained network)
- pretrained network는 일반적으로 대규모 이미지 분류 문제를 위해 대량의 데이터셋에서 미리 훈련된 네트워크를 뜻함
- pretrained network를 사용하는 것은 작은 이미지 데이터셋에 딥러닝을 적용하는 일반적이고 효과적인 방법


- **일반적이고 충분히 큰 데이터셋에 훈련된 네트워크의 특성 계층 구조는 실제 세상에 대한 일반적인 모델로 효율적인 역할을 할 수 있음**
- 위와 같은 유연성은 얕은 학습과 비교해서 딥러닝의 핵심 장점이라고 할 수 있음
- 이런 특성은 완전히 다른 클래스의 경우라도 많은 컴퓨터 비전 문제에서 유용함


- 여기서는 ImageNet 데이터셋으로 학습된 VGG16 모델을 사용
    - ImageNet Dataset은 1400만개의 이미지와 1000개의 class를 가짐
    - VGG16 말고도 다른 많은 모델들에도 적용이 가능하지만 간단해서 사용
  
  
- `keras.applications` 모듈
    - 여러 이미지 분류 모델을 import해서 사용할 수 있음(이 절에서 VGG16 모델을 가져올 때 사용)
    - 사용 가능한 여러 모델이 있음(모두 ImageNet에서 훈련된 모델)
        - Xception
        - Inception V3
        - ResNet50
        - VGG16
        - VGG19
        - MobileNet
    

## Pretrained network 사용 방법 2가지
1. **특성 추출(Feature Extraction)**
2. **미세 조정(Fine Tuning)**

## 5.3.1 특성 추출 (Feature Extraction)

### 특성 추출
- 특성 추출은 사전에 학습된 네트워크의 표현을 사용해 새로운 샘플의 특성을 뽑아내는 것


### Convnet의 특성 추출
- Convnet의 구성
    - 합성곱 기반 층
        - Convolutional layer
        - Pooling layer
    - 완전 연결 분류기
        - Fully connected layer


- 1번째 합성곱 기반 층은 동결시킨 후 재사용
- 2번째 완전 연결 분류기는 새로 학습시킴
- 따라서 **동결된 합성곱 기반 층의 출력을 이용해 2번째 완전 연결 분류기를 새로 학습시키는 방식**


### 합성곱 층만 재사용 하는 이유
- **합성곱 층에 의해 학습된 표현이 더 일반적이므로** 여러 컴퓨터비전 문제에서 유용하게 사용 가능함
- convnet의 feature map은 사진에 대해 일반적인 컨셉의 존재 여부를 기록한 맵임
- **완전 연결 층은 어떤 클래스가 존재할 확률에 관한 정보만 담고있으므로 클래스가 겹치지 않는 경우에는 재사용 하지 않음**
- 참고) ImageNet에는 고양이와 강아지가 있어서 완전 연결 층도 재사용하는 것이 더 도움이 되지만 새로운 문제의 클래스가 원본 모델의 클래스와 겹치지 않는 일반적인 경우를 다루기 위해 사용하지 않았음


### 합성곱 층의 추출 정도
- **합성곱 층에서 추출한 표현의 일반성(재사용성)은 모델 층의 깊이에 달려있음**
- 모델의 **하위 층은 지역적(엣지, 색깔, 질감 등)이고 일반적인 특성맵**으로 구성됨
- 모델의 **상위 층은 고수준(강아지 눈이나 고양이 귀 등)의 추상적인 특성맵**으로 구성됨
- **사용할 데이터 셋이 원본 모델에 사용된 데이터 셋과 많이 다르다면 전체 합성곱 층을 재사용 하는 것 보다는 하위 층 일부만 추출해 사용하는 것이 좋음**

In [1]:
# VGG16 합성곱 층 가져오기

from keras.applications import VGG16

# VGG16()의 매개변수들
#
# weights : 초기화할 가중치 체크포인트
#           None 지정 시 random initialization
#           'imagenet' 지정 시 imagenet에 pretrained된 가중치
#
# include_top : 최상위 완전 연결 분류기의 포함 여부
#
# input_shape : 네트워크에 주입할 이미지 텐서 크기
#               이 값을 지정하지 않으면 어떤 크기의 입력도 처리할 수 있음
#               include_top이 True였다면 완전 연결 분류기가 추가되므로
#               원본 모델과 동일한 (224, 224, 3)이 되어야 함

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150,150, 3))

conv_base.summary()

Using TensorFlow backend.


A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 6d6bbae143d832006294945121d1f1fc so we will re-download the data.
58892288/58889256 [==============================] - 10s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856   